In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#Record 불러와서 DataFrame으로 변환
df=pd.read_csv('naver_blog_crawl_merged_results_noAD.csv')
print('record number',len(df))

record number 4025


In [3]:
#결측치 제거 (내용, 문맥에 관련한 전처리)
context = df[['내용']]
df1= context.dropna()
print('record number',len(df1))

print(df1)

# '내용' 열을 리스트로 변환
context_list = df1['내용'].tolist()


#확인용
#print(context_list[0])

record number 4013
                                                     내용
0     100% 내돈내산 솔직 후 기\n오사카 도톤보리 츠케멘 맛집 "스즈메"\n내돈내산으...
1     바다뷰 루프탑 카페\n다츠도츠\n🚩위치 주소 : 인천 중구 대무의로 230 지하1,...
2     이탈리아 일주하면서 가장 좋았던 도시를 꼽으라면?\n밀라노, 베네치아, 로마, 또 ...
3     [수원가정의달맛집]\n명륜진사갈비 수원월드컵점\n다양한 메뉴 넓은매장 주차정보 내돈...
4     서울 시청역 맛집 퍼스트플로어 센트럴\n내돈내산 맛집\n서울 시청역에 있는\n프레이...
...                                                 ...
4020  9월 9일 월요일\n#가로수길데이트 #가로수길 #가로수길데이트코스 #신사동데이트\n...
4021  포토로그\n서비스 점검 안내 5월 23일(목) 오전 1시~6시 정기점검이 진행됩니다...
4022  블로그\n서비스 점검 안내 5월 23일(목) 오전 1시~6시 정기점검이 진행됩니다....
4023  3월 마지막과 4월 1,2\n3월을 잘 마무리할 자신이 없어 미루고 미루다 이렇게 ...
4024  초등학교 때 친구들이랑 오랜만에 만나기로 했는데 어디 갈지 못 정하고 있었는데, 면...

[4013 rows x 1 columns]


In [4]:
#결측치 제거 (제목은 키워드가 많이 들어갈 수 밖에 없다.->임펙트 중요하므로 파악 필요)
title = df[['제목']]
df2= title.dropna()
print('record number',len(df2))

print(df2)

# '제목' 열을 리스트로 변환
title_list = df2['제목'].tolist()

record number 4023
                                       제목
0        [오사카 도톤보리 맛집] 츠케멘 스즈메 내돈내산 추천 후기
1          무의도 카페 바다뷰가 멋있는 다츠도츠 내돈내산 솔직후기
2     투어 내돈내산 솔직후기, 피렌체 여행, 이탈리아 투어 추천...
3      [수원가정의달맛집]... 메뉴 넓은 매장 주차정보 내돈내산리뷰
4            서울 시청역 맛집 퍼스트플로어 센트럴 내돈내산 맛집
...                                   ...
4020    평일 가로수길 데이트 맛집 솔직후기/도산분식/숲숨그레이...
4021                   경제적 자유를 위한 부자 마인드셋
4022      대만, 베트남, 라오스, 태국 4개국 배낭여행  - 1탄
4023                         맛집탐방 및 사진정리7
4024           수많은 인파 속에서 나는 한눈에 널 알아본 거야

[4023 rows x 1 columns]


In [5]:
import re
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

def preprocess_korean_text(text):
    # 텍스트 정제: 특수 문자, 숫자, 영어 등 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    
    # 토큰화 및 품사 태깅
    tokens = okt.pos(text, stem=True)
    
    # 불용어 목록 (예시)
    stop_words = set(['그리고', '그래서', '그러나', '하지만', '또한', '및', '이것', '저것'])
    
    # 불용어 제거
    processed_tokens = [(word, pos) for word, pos in tokens if word not in stop_words and len(word) > 1]
    
    # 결과 확인을 위한 출력
    print("전처리된 토큰 결과:")
    print(processed_tokens)
    
    # 전처리된 텍스트 반환
    processed_text = ' '.join([word for word, pos in processed_tokens])
    return processed_text

# 예시 텍스트(test용)
#text = "이 텍스트는 한국어 전처리를 위한 예시 문장입니다. 다양한 처리 과정을 거칩니다!"

# 전처리된 텍스트를 저장할 리스트
preprocessed_texts = []

# 리스트의 각 요소에 대해 전처리 함수를 적용하여 저장
for text in title_list:
    preprocessed_text = preprocess_korean_text(text)
    preprocessed_texts.append(preprocessed_text)

# 결과 확인
print(preprocessed_texts)



전처리된 토큰 결과:
[('오사카', 'Noun'), ('도톤보리', 'Noun'), ('맛집', 'Noun'), ('츠케멘', 'Noun'), ('스즈메', 'Noun'), ('추천', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('무의도', 'Noun'), ('카페', 'Noun'), ('바다', 'Noun'), ('멋있다', 'Adjective'), ('다츠', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('투어', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun'), ('피렌체', 'Noun'), ('여행', 'Noun'), ('이탈리아', 'Noun'), ('투어', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('가정', 'Noun'), ('맛집', 'Noun'), ('메뉴', 'Noun'), ('넓다', 'Adjective'), ('매장', 'Noun'), ('주차', 'Noun'), ('정보', 'Noun'), ('리뷰', 'Noun')]
전처리된 토큰 결과:
[('서울', 'Noun'), ('시청역', 'Noun'), ('맛집', 'Noun'), ('퍼스트', 'Noun'), ('플로', 'Noun'), ('센트럴', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('제주도', 'Noun'), ('중문', 'Noun'), ('맛집', 'Noun'), ('제주', 'Noun'), ('밉다', 'Adjective'), ('점심', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('오사카', 'Noun'), ('맛집', 'Noun'), ('난바역', 'Noun'), ('에서', 'Josa'), ('만두', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('오이도', 'Noun'), ('조개', 'Noun'), ('구이', 'Noun'), 

전처리된 토큰 결과:
[('극찬', 'Noun'), ('콩물', 'Noun'), ('맛집', 'Noun'), ('전주', 'Noun'), ('베테', 'Noun'), ('칼국수', 'Noun'), ('콩물', 'Noun'), ('택배', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('솔직', 'Noun'), ('후기', 'Noun'), ('비빔', 'Noun'), ('메밀국수', 'Noun'), ('메밀전', 'Noun')]
전처리된 토큰 결과:
[('당진', 'Noun'), ('해물', 'Noun'), ('칼국수', 'Noun'), ('대성', 'Noun'), ('포장마차', 'Noun'), ('솔직', 'Noun'), ('방문', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('과천', 'Noun'), ('선바위', 'Noun'), ('맛집', 'Noun'), ('메밀', 'Noun'), ('장터', 'Noun'), ('들기름', 'Noun'), ('막국수', 'Noun'), ('파전', 'Noun')]
전처리된 토큰 결과:
[('인사동', 'Noun'), ('맛집', 'Noun'), ('인사동', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('흥동', 'Noun'), ('점심', 'Noun'), ('맛집', 'Noun'), ('공기밥', 'Noun'), ('무한리필', 'Noun'), ('완벽', 'Noun'), ('자체', 'Noun')]
전처리된 토큰 결과:
[('강원', 'Noun'), ('속초', 'Noun'), ('맛집', 'Noun'), ('장터', 'Noun'), ('대국', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('일산', 'Noun'), ('단체', 'Noun'), ('모임', 'Noun'), ('경복궁', 'Noun'), ('화점', 'Noun'), 

전처리된 토큰 결과:
[('유럽', 'Noun'), ('세미', 'Noun'), ('패키지', 'Noun'), ('비교', 'Noun'), ('정리', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('서울', 'Noun'), ('근교', 'Noun'), ('카페', 'Noun'), ('마운틴뷰', 'Noun'), ('자연', 'Noun'), ('카페', 'Noun'), ('나크', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('소고기', 'Noun'), ('괜찮다', 'Adjective'), ('맛집', 'Noun'), ('이라는', 'Josa'), ('원조', 'Noun'), ('갈비살', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('모으다', 'Verb'), ('평점', 'Noun'), ('인천', 'Noun'), ('부평', 'Noun'), ('맛집', 'Noun'), ('추억', 'Noun'), ('빙수', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('부산', 'Noun'), ('씨앗', 'Noun'), ('호떡', 'Noun'), ('맛집', 'Noun'), ('별난', 'Noun'), ('호떡', 'Noun')]
전처리된 토큰 결과:
[('근처', 'Noun'), ('숙소', 'Noun'), ('추천', 'Noun'), ('좋다', 'Adjective'), ('방콕', 'Noun'), ('아속', 'Noun'), ('호텔', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('봉명동', 'Noun'), ('고기', 'Noun'), ('초벌', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('부산', 'Noun'), ('사상', 'Noun'), ('밥집', 'Noun'), ('솟다', 'Verb'), ('해물', 'Noun'

전처리된 토큰 결과:
[('맛집', 'Noun'), ('고든', 'Noun'), ('램지', 'Noun'), ('스트리트', 'Noun'), ('피자', 'Noun'), ('메뉴', 'Noun'), ('추천', 'Noun'), ('솔직', 'Noun')]
전처리된 토큰 결과:
[('하남', 'Noun'), ('미사', 'Noun'), ('포케', 'Noun'), ('샐러드', 'Noun'), ('맛집', 'Noun'), ('아보', 'Noun'), ('포케', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('대전', 'Noun'), ('관평동', 'Noun'), ('맛집', 'Noun'), ('대패', 'Noun'), ('방문', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('대산', 'Noun'), ('보리밥', 'Noun'), ('고등어', 'Noun'), ('피자', 'Noun'), ('주다', 'Verb'), ('성화동', 'Noun'), ('한식', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('부천', 'Noun'), ('아이', 'Noun'), ('네일', 'Noun'), ('아트', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('합정', 'Noun'), ('물고기', 'Noun'), ('초밥', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('서대문구', 'Noun'), ('대형', 'Noun'), ('카페', 'Noun'), ('바스크', 'Noun'), ('치즈케이크', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('석남', 'Noun'), ('일본', 'Noun'), ('느낌', 'Noun'), ('물씬', 'Noun'), ('이자카야', 'Noun'), ('타키', 'Noun'), 

전처리된 토큰 결과:
[('솔직', 'Noun'), ('후기', 'Noun'), ('수제', 'Noun'), ('버거', 'Noun'), ('미국', 'Noun'), ('버거', 'Noun'), ('논현', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('마곡나루역', 'Noun'), ('카페', 'Noun'), ('리프', 'Noun')]
전처리된 토큰 결과:
[('카페', 'Noun'), ('후기', 'Noun'), ('우연히', 'Adverb'), ('들어가다', 'Verb'), ('디저트', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('판교', 'Noun'), ('인생', 'Noun'), ('곱창', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('한방약', 'Noun'), ('오리', 'Noun'), ('불고기', 'Noun'), ('싶다', 'Verb'), ('오리고기', 'Noun'), ('찌다', 'Verb'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('홍대', 'Noun'), ('연남', 'Noun'), ('취향', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun'), ('웨이', 'Noun')]
전처리된 토큰 결과:
[('오산', 'Noun'), ('맛집', 'Noun'), ('육미', 'Noun'), ('부모님', 'Noun'), ('장소', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('맛집', 'Noun'), ('타코', 'Noun'), ('데이', 'Noun'), ('웨이', 'Noun'), ('주차', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('복코', 'Noun'), ('조각', 'Noun'), ('주사', 'Noun'), ('콧볼', 'Noun'), ('보톡스', 'Noun'), ('솔직

...

전처리된 토큰 결과:
[('일상', 'Noun'), ('기록', 'Noun'), ('명동성당', 'Noun'), ('맛집', 'Noun'), ('광장동', 'Noun'), ('가온', 'Noun'), ('르빵홍밀밀', 'Noun'), ('속초', 'Noun')]
전처리된 토큰 결과:
[('오월', 'Noun'), ('푸르다', 'Adjective'), ('구나', 'Noun'), ('우리', 'Noun'), ('자라다', 'Verb')]
전처리된 토큰 결과:
[('먹다', 'Verb'), ('보다', 'Verb'), ('물회', 'Noun'), ('청초', 'Noun'), ('물회', 'Noun'), ('일산', 'Noun'), ('맛집', 'Noun'), ('물회', 'Noun'), ('첫경험', 'Noun')]
전처리된 토큰 결과:
[('제주', 'Noun'), ('맛집', 'Noun'), ('식당', 'Noun'), ('고등어', 'Noun'), ('조림', 'Noun'), ('최고', 'Noun')]
전처리된 토큰 결과:
[('발렌타인', 'Noun'), ('초코', 'Noun'), ('브라우니', 'Noun'), ('커피', 'Noun'), ('강남', 'Noun'), ('포스코', 'Noun'), ('사거리', 'Noun'), ('점내', 'Noun')]
전처리된 토큰 결과:
[('대전', 'Noun'), ('도마동', 'Noun'), ('맛집', 'Noun'), ('여름', 'Noun'), ('콩국수', 'Noun'), ('맛집', 'Noun'), ('대성', 'Noun'), ('콩국수', 'Noun')]
전처리된 토큰 결과:
[('일본', 'Noun'), ('여행', 'Noun'), ('맛집', 'Noun'), ('투어', 'Noun'), ('직접', 'Noun'), ('가보다', 'Verb'), ('일본', 'Noun'), ('맛집', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('춘천', 'Noun'), ('소양강',

전처리된 토큰 결과:
[('금능', 'Noun'), ('해수욕장', 'Noun'), ('맛집', 'Noun'), ('없이', 'Adverb'), ('먹다', 'Verb'), ('갈치', 'Noun'), ('오다', 'Verb'), ('오다', 'Verb'), ('국수', 'Noun')]
전처리된 토큰 결과:
[('전주', 'Noun'), ('간장게장', 'Noun'), ('맛집', 'Noun'), ('직접', 'Noun'), ('담그다', 'Verb'), ('간장게장', 'Noun'), ('대감', 'Noun')]
전처리된 토큰 결과:
[('음식점', 'Noun'), ('광고', 'Noun'), ('맛집', 'Noun'), ('되다', 'Verb'), ('필수', 'Noun'), ('전략', 'Noun'), ('필독', 'Noun')]
전처리된 토큰 결과:
[('전남', 'Noun'), ('곡성', 'Noun'), ('기차', 'Noun'), ('마을', 'Noun'), ('맛집', 'Noun'), ('별천지', 'Noun'), ('가든', 'Noun')]
전처리된 토큰 결과:
[('성남', 'Noun'), ('마케팅', 'Noun'), ('회사', 'Noun'), ('맛집', 'Noun'), ('광고', 'Noun'), ('효과', 'Noun'), ('으로', 'Josa')]
전처리된 토큰 결과:
[('맛집', 'Noun'), ('블로그', 'Noun'), ('광고', 'Noun'), ('대행', 'Noun'), ('관리', 'Noun'), ('제대로', 'Noun'), ('받다', 'Verb'), ('싶다', 'Verb')]
전처리된 토큰 결과:
[('식당', 'Noun'), ('광고', 'Noun'), ('맛집', 'Noun'), ('파워블로거', 'Noun'), ('함께', 'Adverb'), ('시너지', 'Noun'), ('높이다', 'Verb')]
전처리된 토큰 결과:
[('변호사', 'Noun'), ('광고', 'Noun'), ('규정', 'No

전처리된 토큰 결과:
[('방이', 'Noun'), ('먹다', 'Verb'), ('골목', 'Noun'), ('인스타', 'Noun'), ('광고', 'Noun'), ('맛집', 'Noun'), ('방이', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('네이버', 'Noun'), ('블로그', 'Noun'), ('광고', 'Noun'), ('실패', 'Noun'), ('없이', 'Adverb'), ('진행', 'Noun'), ('하다', 'Verb')]
전처리된 토큰 결과:
[('신길역', 'Noun'), ('치과', 'Noun'), ('광고', 'Noun'), ('없이', 'Adverb'), ('운영', 'Noun'), ('하다', 'Verb'), ('치과', 'Noun'), ('찾다', 'Verb'), ('방법', 'Noun'), ('공개', 'Noun')]
전처리된 토큰 결과:
[('문경', 'Noun'), ('현지', 'Noun'), ('맛집', 'Noun'), ('산나물', 'Noun'), ('예약', 'Noun'), ('없이', 'Adverb'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('대구', 'Noun'), ('엘리베이터', 'Noun'), ('광고', 'Noun'), ('하다', 'Verb'), ('어떤', 'Modifier'), ('중요하다', 'Adjective'), ('체크', 'Noun'), ('하다', 'Verb')]
전처리된 토큰 결과:
[('나트랑', 'Noun'), ('푸드', 'Noun'), ('맛집', 'Noun'), ('뜨다', 'Verb'), ('해산물', 'Noun'), ('스타', 'Noun'), ('솔직', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('세종시', 'Noun'), ('고운동', 'Noun'), ('맛집', 'Noun'), ('추천', 'Noun'), ('직접', 'Noun'), ('만들다', 'Verb'),

전처리된 토큰 결과:
[('다산', 'Noun'), ('피자', 'Noun'), ('맛집', 'Noun'), ('번가', 'Noun'), ('피자', 'Noun'), ('에서', 'Josa'), ('테이크아웃', 'Noun')]
전처리된 토큰 결과:
[('포항', 'Noun'), ('일대', 'Modifier'), ('맛집', 'Noun'), ('칼칼하다', 'Adjective'), ('횟집', 'Noun'), ('웨이', 'Noun'), ('없이', 'Adverb'), ('들어가다', 'Verb'), ('주차', 'Noun')]
전처리된 토큰 결과:
[('대구', 'Noun'), ('동구', 'Noun'), ('맛집', 'Noun'), ('팔공산', 'Noun'), ('어버이날', 'Noun'), ('식당', 'Noun'), ('가족', 'Noun'), ('모임', 'Noun'), ('명산', 'Noun'), ('한우', 'Noun'), ('식당', 'Noun')]
전처리된 토큰 결과:
[('사이판', 'Noun'), ('가라', 'Noun'), ('맛집', 'Noun'), ('남대문', 'Noun'), ('코코', 'Noun'), ('레스토랑', 'Noun'), ('추천', 'Noun'), ('무료', 'Noun'), ('쿠폰', 'Noun')]
전처리된 토큰 결과:
[('부평', 'Noun'), ('맛집', 'Noun'), ('살다', 'Verb'), ('오징어', 'Noun'), ('직접', 'Noun'), ('잡다', 'Verb'), ('요리', 'Noun'), ('하다', 'Verb'), ('오징어', 'Noun'), ('슈퍼', 'Noun')]
전처리된 토큰 결과:
[('화곡동', 'Noun'), ('맛집', 'Noun'), ('당일', 'Noun'), ('도축', 'Noun'), ('한우', 'Noun'), ('곱창', 'Noun'), ('오다', 'Verb'), ('가다', 'Verb'), ('황소', 'Noun'), ('곱창', 'Noun')

전처리된 토큰 결과:
[('사이클링', 'Noun'), ('의류', 'Noun'), ('마빅', 'Noun'), ('코스', 'Noun'), ('얼티', 'Noun'), ('메이트', 'Noun'), ('빕숏', 'Noun'), ('협찬', 'Noun'), ('광고', 'Noun')]
전처리된 토큰 결과:
[('순하다', 'Adjective'), ('또또', 'Noun'), ('잠실역', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('네이버', 'Noun'), ('카페', 'Noun'), ('광고', 'Noun'), ('매출', 'Noun'), ('올리다', 'Verb'), ('비결', 'Noun')]
전처리된 토큰 결과:
[('말다', 'Verb'), ('발산', 'Noun'), ('역점', 'Noun'), ('정갈하다', 'Adjective'), ('제철', 'Noun'), ('재료', 'Noun'), ('즐기다', 'Verb'), ('가성', 'Noun'), ('좋다', 'Adjective'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('대구', 'Noun'), ('서구청', 'Noun'), ('맛집', 'Noun'), ('팔일', 'Noun'), ('고기', 'Noun'), ('잊다', 'Verb'), ('평리동', 'Noun'), ('고깃집', 'Noun')]
전처리된 토큰 결과:
[('교토', 'Noun'), ('소바', 'Noun'), ('맛집', 'Noun'), ('소바', 'Noun'), ('노미', 'Noun'), ('시무', 'Noun'), ('라니', 'Josa'), ('조성', 'Noun'), ('근처', 'Noun'), ('아쉽다', 'Adjective')]
전처리된 토큰 결과:
[('강화도', 'Noun'), ('맛집', 'Noun'), ('천원', 'Noun'), ('행복', 'Noun')]
전처리된 토큰 결과:
[('병원', 'Noun'), ('블로그', 'Noun'), ('광고', 'Noun

In [6]:
def detokenize_text(tokens):
    # 토큰 사이에 공백을 추가하여 문장으로 결합
    return ''.join(tokens)

# 전처리된 토큰에 역토큰화 함수를 적용하여 문서 형식으로 반환
processed_documents = pd.Series(preprocessed_texts).apply(lambda x: detokenize_text(x))

# 결과 확인
print(processed_documents)

0               오사카 도톤보리 맛집 츠케멘 스즈메 추천 후기
1                  무의도 카페 바다 멋있다 다츠 솔직 후기
2              투어 솔직 후기 피렌체 여행 이탈리아 투어 추천
3                 가정 맛집 메뉴 넓다 매장 주차 정보 리뷰
4                 서울 시청역 맛집 퍼스트 플로 센트럴 맛집
                      ...                
4018    평일 가로수길 데이트 맛집 솔직 후기 도산 분식 숲숨 그레이
4019                         경제 자유 부자 마인드
4020             대만 베트남 라오스 태국 개국 배낭여행 타다
4021                          맛집 탐방 사진 정리
4022                 수많다 인파 에서 한눈 알아보다 거야
Length: 4023, dtype: object
